<a href="https://colab.research.google.com/github/navrat/semi_sup_learning_text_clf/blob/main/Semi_Supervised_self_training_sklearn_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
import torch.utils.data as Data
import pickle

In [3]:
print(np.__version__)
import sklearn
print(sklearn.__version__)

1.21.6
1.0.2


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# !pip install numpy==1.16.2
# import numpy as np

In [6]:
import pandas as pd

In [7]:
# !pip install pandas==1.2.1
# import numpy as np

In [8]:
# !pip install --upgrade scikit-learn  # Do this to use sklearn SelfTrainingClassifier

In [9]:
# import numpy as np
# np.__version__

In [10]:
# %cd "/content/drive/My Drive/Colab Notebooks/MixText-master"
# !ls

In [11]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


In [12]:

data = fetch_20newsgroups(subset='train', categories=None)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()

# Parameters
sdg_params = dict(alpha=1e-5, penalty='l2', loss='log', random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

# Supervised Pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(**sdg_params)),
])

# SelfTraining Pipeline
st_pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    ('clf', SelfTrainingClassifier(SGDClassifier(**sdg_params), verbose=True)),
])


def eval_and_print_metrics(clf, X_train, y_train, X_test, y_test):
    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))
    print("-" * 10)
    print()




11314 documents
20 categories



In [13]:
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
type(y_train)

numpy.ndarray

In [ ]:
# SGD - Log Loss(Logistic regression).
if __name__ == "__main__":

    n_list = [10, 20, 30, 40, 50]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = "+str(n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
      X, y = data.data, data.target
      X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)



      unlabeled_mask = np.random.rand(len(y_train)) < 0.5
      X_u50, y_u50 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_train, y_train, unlabeled_mask) if m)))
      
      y_u50 = np.array([-1 for i in y_u50])

      X_50, y_50 = map(list, zip(*((x, y)
                for x, y, m in zip(X_train, y_train, unlabeled_mask) if ~m)))
      




      percentage = 2*(n/100)
      y_mask = np.random.rand(len(y_50)) < percentage

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_50, y_50, y_mask) if m)))
      print("Supervised SGDClassifier on "+str(n)+"% of the training data:")
      
      eval_and_print_metrics(pipeline, X_20, y_20, X_test, y_test)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(SGDClassifier(**sdg_params), threshold = t, verbose=True)),
        ])
        eval_and_print_metrics(st_pipeline, X_20+X_u50, np.concatenate((y_20, y_u50)), X_test, y_test)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = 10~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised SGDClassifier on 10% of the training data:
Number of training samples: 869
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.709
Accuracy Score:  0.7090844821491693
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training set: 4307
End of iteration 1, added 2569 new labels.
End of iteration 2, added 761 new labels.
End of iteration 3, added 211 new labels.
End of iteration 4, added 69 new labels.
End of iteration 5, added 35 new labels.
End of iteration 6, added 17 new labels.
End of iteration 7, added 14 new labels.
End of iteration 8, added 6 new labels.
End of iteration 9, added 6 new labels.
End of iteration 10, added 5 new labels.
Micro-averaged F1 score on test set

In [14]:
data = fetch_20newsgroups(subset='train', categories=None)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()


def eval_and_print_metrics_df(clf, X_train, y_train, X_test, y_test, thresh = None, kbest = None):

    dict1 = {}


    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    
    dict1['Labeled'] = len(X_train) - sum(1 for x in y_train if x == -1)
    dict1['UnLabeled'] = sum(1 for x in y_train if x == -1)


    
    # if sum(1 for x in y_train if x == -1) == 0:
    #     dict1['type'] = 'Supervised'
    # else:
    #     dict1['type'] = 'Semi-Supervised'

    dict1['Threshold'] = thresh
    dict1['K-Best'] = kbest

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))

    dict1['Accuracy'] = accuracy_score(y_test, y_pred)

    print("-" * 10)
    print()

    return dict1




11314 documents
20 categories



In [15]:
## Running individual sklearn algorithms

In [ ]:
# Naive Bayes
from sklearn.naive_bayes import MultinomialNB

# Parameters
# mnb_params = 
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_nb_ng = pd.DataFrame()

    n_list = [10, 20, 30, 40, 50]
    kbest_list=[4, 5, 6, 7, 8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = "+str(n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
      X, y = data.data, data.target
      X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)



      unlabeled_mask = np.random.rand(len(y_train)) < 0.5
      X_u50, y_u50 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_train, y_train, unlabeled_mask) if m)))
      
      y_u50 = np.array([-1 for i in y_u50])

      X_50, y_50 = map(list, zip(*((x, y)
                for x, y, m in zip(X_train, y_train, unlabeled_mask) if ~m)))
      




      percentage = 2*(n/100)
      y_mask = np.random.rand(len(y_50)) < percentage

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_50, y_50, y_mask) if m)))
      print("Supervised MNB on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', MultinomialNB()),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_nb_ng = df_nb_ng.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for kb in kbest_list:
        kbest = int(len(X_u50)/kb)
        print("---------------------------------K-Best = ", kbest,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(MultinomialNB(), criterion = 'k_best', k_best = kbest, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_20+X_u50, np.concatenate((y_20, y_u50)), X_test, y_test, thresh = None, kbest = kbest)
        df_nb_ng = df_nb_ng.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = 10~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised SGDClassifier on 10% of the training data:
Number of training samples: 869
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.545
Accuracy Score:  0.545068928950159
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------K-Best =  1076 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training set: 4307
End of iteration 1, added 1076 new labels.
End of iteration 2, added 1076 new labels.
End of iteration 3, added 1076 new labels.
End of iteration 4, added 1076 new labels.
End of iteration 5, added 3 new labels.
Micro-averaged F1 score on test set: 0.055
Accuracy Score:  0.055496641922940966
----------

---------------------------------K-Best =  861 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training s

In [ ]:
display(df_nb_ng[['Labeled', 'UnLabeled', 'Threshold', 'K-Best', 'Accuracy']].sort_values(['Labeled', 'UnLabeled']))

,Labeled,UnLabeled,Threshold,K-Best,Accuracy
0,869.0,0.0,NaN,NaN,0.545069
1,869.0,4307.0,NaN,1076.0,0.055497
2,869.0,4307.0,NaN,861.0,0.055143
3,869.0,4307.0,NaN,717.0,0.054436
4,869.0,4307.0,NaN,615.0,0.054083
5,869.0,4307.0,NaN,538.0,0.053376
6,1695.0,0.0,NaN,NaN,0.706257
7,1695.0,4307.0,NaN,1076.0,0.063273
8,1695.0,4307.0,NaN,861.0,0.061152
9,1695.0,4307.0,NaN,717.0,0.060445


In [ ]:
df_nb_ng.pivot(index=['Labeled', 'UnLabeled'], columns='K-Best')['Accuracy']

K-Best               NaN       538.0     615.0     717.0     861.0     1076.0
Labeled UnLabeled                                                            
869.0   0.0        0.545069       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.053376  0.054083  0.054436  0.055143  0.055497
1695.0  0.0        0.706257       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.059738  0.059738  0.060445  0.061152  0.063273
2541.0  0.0        0.742312       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.106045  0.073524  0.074585  0.072464  0.079533
3349.0  0.0        0.770944       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.074231  0.078119  0.082715  0.095794  0.111700
4178.0  0.0        0.799222       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.094026  0.099328  0.105338  0.116649  0.142807

In [ ]:
# Random Forest for NG
from sklearn.ensemble import RandomForestClassifier

# Parameters
# mnb_params = 
rf_params = dict(n_estimators=100, random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_rf_ng = pd.DataFrame()

    n_list = [10, 20, 30, 40, 50]
    kbest_list=[4, 5, 6, 7, 8]
    threshold = [0.4, 0.5, 0.6, 0.7, 0.8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = "+str(n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
      X, y = data.data, data.target
      X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)



      unlabeled_mask = np.random.rand(len(y_train)) < 0.5
      X_u50, y_u50 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_train, y_train, unlabeled_mask) if m)))
      
      y_u50 = np.array([-1 for i in y_u50])

      X_50, y_50 = map(list, zip(*((x, y)
                for x, y, m in zip(X_train, y_train, unlabeled_mask) if ~m)))
      




      percentage = 2*(n/100)
      y_mask = np.random.rand(len(y_50)) < percentage

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_50, y_50, y_mask) if m)))

      print("Supervised NBClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', RandomForestClassifier(**rf_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_rf_ng = df_rf_ng.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(RandomForestClassifier(**rf_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_20+X_u50, np.concatenate((y_20, y_u50)), X_test, y_test, thresh = t, kbest = None)
        df_rf_ng = df_rf_ng.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = 10~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised NBClassifier on 10% of the training data:
Number of training samples: 869
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.654
Accuracy Score:  0.6535878402262284
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training set: 4307
End of iteration 1, added 634 new labels.
End of iteration 2, added 421 new labels.
End of iteration 3, added 476 new labels.
End of iteration 4, added 724 new labels.
End of iteration 5, added 842 new labels.
End of iteration 6, added 877 new labels.
End of iteration 7, added 266 new labels.
End of iteration 8, added 57 new labels.
End of iteration 9, added 10 new labels.
Micro-averaged F1 score on test set: 0.198
Accuracy Score:  0.1975963237

In [ ]:
df_rf_ng.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

Threshold               NaN       0.4       0.5       0.6       0.7       0.8
Labeled UnLabeled                                                            
869.0   0.0        0.653588       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.197596  0.123719  0.253800  0.574054  0.645811
1695.0  0.0        0.725345       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.275362  0.341110  0.654295  0.705196  0.726405
2541.0  0.0        0.758925       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.578296  0.674797  0.722517  0.738423  0.747967
3349.0  0.0        0.783669       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.728526  0.734182  0.744079  0.765642  0.780134
4178.0  0.0        0.791092       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.765288  0.772711  0.770944  0.790032  0.793213

In [ ]:
## SGD (Logistic Regression) experiment

In [ ]:
# Logistic Regression
from sklearn.linear_model import SGDClassifier

# Parameters
sgd_params = dict(alpha=1e-5, penalty='l2', loss='log', random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

if __name__ == "__main__":

    df_sgd_ng = pd.DataFrame()

    n_list = [10, 20, 30, 40, 50]
    kbest_list=[4, 5, 6, 7, 8]
    threshold = [0.4, 0.5, 0.6, 0.7, 0.8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = "+str(n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
      X, y = data.data, data.target
      X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)



      unlabeled_mask = np.random.rand(len(y_train)) < 0.5
      X_u50, y_u50 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_train, y_train, unlabeled_mask) if m)))
      
      y_u50 = np.array([-1 for i in y_u50])

      X_50, y_50 = map(list, zip(*((x, y)
                for x, y, m in zip(X_train, y_train, unlabeled_mask) if ~m)))
      




      percentage = 2*(n/100)
      y_mask = np.random.rand(len(y_50)) < percentage

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_50, y_50, y_mask) if m)))

      print("Supervised SGDClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', SGDClassifier(**sgd_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_sgd_ng = df_sgd_ng.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(SGDClassifier(**sgd_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_20+X_u50, np.concatenate((y_20, y_u50)), X_test, y_test, thresh = t, kbest = None)
        df_sgd_ng = df_sgd_ng.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = 10~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised SGDClassifier on 10% of the training data:
Number of training samples: 869
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.709
Accuracy Score:  0.7090844821491693
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training set: 4307
End of iteration 1, added 2569 new labels.
End of iteration 2, added 761 new labels.
End of iteration 3, added 211 new labels.
End of iteration 4, added 69 new labels.
End of iteration 5, added 35 new labels.
End of iteration 6, added 17 new labels.
End of iteration 7, added 14 new labels.
End of iteration 8, added 6 new labels.
End of iteration 9, added 6 new labels.
End of iteration 10, added 5 new labels.
Micro-averaged F1 score on test set

In [ ]:
df_sgd_ng

,Labeled,UnLabeled,Threshold,K-Best,Accuracy
0,869.0,0.0,NaN,NaN,0.709084
1,869.0,4307.0,0.4,NaN,0.795688
2,869.0,4307.0,0.5,NaN,0.793920
3,869.0,4307.0,0.6,NaN,0.795688
4,869.0,4307.0,0.7,NaN,0.782609
5,869.0,4307.0,0.8,NaN,0.763874
6,1695.0,0.0,NaN,NaN,0.806999
7,1695.0,4307.0,0.4,NaN,0.848003
8,1695.0,4307.0,0.5,NaN,0.853659
9,1695.0,4307.0,0.6,NaN,0.849417


In [ ]:
df_sgd_ng.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

Threshold               NaN       0.4       0.5       0.6       0.7       0.8
Labeled UnLabeled                                                            
869.0   0.0        0.709084       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.795688  0.793920  0.795688  0.782609  0.763874
1695.0  0.0        0.806999       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.848003  0.853659  0.849417  0.847296  0.835277
2541.0  0.0        0.836338       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.868858  0.869565  0.867798  0.860021  0.856486
3349.0  0.0        0.858254       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.885118  0.882998  0.879463  0.872747  0.868505
4178.0  0.0        0.869919       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.895369  0.896430  0.889360  0.887593  0.882998

Clear improvement in accuracies (pivot above) and micro-F1 (console output) can be seen upon using self training vs. training only on labeled dataset

In [ ]:
## MLP Classifier 

In [18]:
# MLP for NG
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Parameters
mlp_params = dict(hidden_layer_sizes=(100,), max_iter=100,activation = 'relu',solver='lbfgs',random_state=1,learning_rate_init=0.01,
                  learning_rate='adaptive') # regularization is by default based on alpha =0.0001
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_mlp_ng = pd.DataFrame()

    n_list = [10, 20, 30, 40, 50]
    kbest_list=[4, 5, 6, 7, 8]
    threshold = [0.4, 0.5, 0.6, 0.7, 0.8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = "+str(n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
      X, y = data.data, data.target
      X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

      # print("Supervised MLPClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)



      unlabeled_mask = np.random.rand(len(y_train)) < 0.5
      X_u50, y_u50 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_train, y_train, unlabeled_mask) if m)))
      
      y_u50 = np.array([-1 for i in y_u50])

      X_50, y_50 = map(list, zip(*((x, y)
                for x, y, m in zip(X_train, y_train, unlabeled_mask) if ~m)))
      




      percentage = 2*(n/100)
      y_mask = np.random.rand(len(y_50)) < percentage

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = map(list, zip(*((x, y)
                      for x, y, m in zip(X_50, y_50, y_mask) if m)))

      print("Supervised MLPClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline      
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('scale', StandardScaler(with_mean=False)),
          ('clf', MLPClassifier(**mlp_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_mlp_ng = df_mlp_ng.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('scale', StandardScaler(with_mean=False)),
            ('clf', SelfTrainingClassifier(MLPClassifier(**mlp_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_20+X_u50, np.concatenate((y_20, y_u50)), X_test, y_test, thresh = t, kbest = None)
        df_mlp_ng = df_mlp_ng.append(temp, ignore_index=True)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~NewsGroup DATA with percentage_labeled = 10~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised MLPClassifier on 10% of the training data:
Number of training samples: 869
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.516
Accuracy Score:  0.5164369034994698
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 5176
Unlabeled samples in training set: 4307
End of iteration 1, added 1676 new labels.
End of iteration 2, added 1256 new labels.
End of iteration 3, added 556 new labels.
End of iteration 4, added 365 new labels.
End of iteration 5, added 127 new labels.
End of iteration 6, added 75 new labels.
End of iteration 7, added 30 new labels.
End of iteration 8, added 21 new labels.
End of iteration 9, added 7 new labels.
End of iteration 10, added 33 new labels.
Micro-averaged F1 score on tes

In [19]:
df_mlp_ng.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']


Threshold               NaN       0.4       0.5       0.6       0.7       0.8
Labeled UnLabeled                                                            
869.0   0.0        0.516437       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.560622  0.491340  0.503005  0.527041  0.507600
1695.0  0.0        0.645458       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.682220  0.685048  0.684341  0.664546  0.659244
2541.0  0.0        0.715447       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.717568  0.737363  0.755744  0.723224  0.707671
3349.0  0.0        0.747614       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.767762  0.767055  0.767409  0.762814  0.769883
4178.0  0.0        0.761753       NaN       NaN       NaN       NaN       NaN
        4307.0          NaN  0.788971  0.802050  0.792506  0.784730  0.785790

Even for an MLP, the same situation holds true. Micro-F1 and Accuracy show improvements using selftrainingclassifier and usually the higher tresholds result in better performance.

In [ ]:
## End of relevant notebook reference - Logistic Regression and Multi-layer perceptron based self training classifiers.

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
def train_val_split(labels, n_labeled_per_class, unlabeled_per_class, n_labels, seed=0):
    """Split the original training set into labeled training set, unlabeled training set, development set

    Arguments:
        labels {list} -- List of labeles for original training set
        n_labeled_per_class {int} -- Number of labeled data per class
        unlabeled_per_class {int} -- Number of unlabeled data per class
        n_labels {int} -- The number of classes

    Keyword Arguments:
        seed {int} -- [random seed of np.shuffle] (default: {0})

    Returns:
        [list] -- idx for labeled training set, unlabeled training set, development set
    """
    np.random.seed(seed)
    labels = np.array(labels)
    train_labeled_idxs = []
    train_unlabeled_idxs = []
    val_idxs = []

    for i in range(n_labels):
        idxs = np.where(labels == i)[0]
        np.random.shuffle(idxs)
        if n_labels == 2:
            # IMDB
            
            
            
            n_unlabeled_per_class = unlabeled_per_class   #10, 100, 500, 1000, 2500
            train_pool = np.concatenate((idxs[:500], idxs[5500:-2000]))
            train_labeled_idxs.extend(train_pool[:n_labeled_per_class])
            train_unlabeled_idxs.extend(idxs[500: 500 + n_unlabeled_per_class])
            val_idxs.extend(idxs[-2000:])
        
        
        
            # train_pool = np.concatenate((idxs[:500], idxs[5500:-2000]))
            # train_labeled_idxs.extend(train_pool[:n_labeled_per_class])
            # train_unlabeled_idxs.extend(
            #     idxs[500: 500 + 5000])
            # val_idxs.extend(idxs[-2000:])
            

    return train_labeled_idxs, train_unlabeled_idxs, val_idxs



import re
#Removes Punctuations
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data



def clean(train_df):
    
    train_df['review']=train_df['review'].apply(lambda z: remove_punctuations(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_html(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_url(z))
    # train_df['review']=train_df['review'].apply(lambda z: remove_emoji(z))
    
    train_df['review']=train_df['review'].apply(lambda z: word_tokenize(z))
    
    # lemmatizer = WordNetLemmatizer()
    # train_df['review']=train_df['review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    train_df['review']=train_df['review'].apply(lambda x: ' '.join(x))
    
    return train_df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import os

import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report



# Parameters
sdg_params = dict(alpha=1e-5, penalty='l2', loss='log', random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

# Supervised Pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(**sdg_params)),
])

# # Supervised Pipeline
# pipeline = Pipeline([
#     ('vect', CountVectorizer(**vectorizer_params)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', SVC(probability=True, gamma="auto")),
# ])

# SelfTraining Pipeline
st_pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    ('clf', SelfTrainingClassifier(SGDClassifier(**sdg_params), verbose=True)),
])

# # SelfTraining Pipeline
# st_pipeline = Pipeline([
#     ('vect', CountVectorizer(**vectorizer_params)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', SelfTrainingClassifier(SVC(probability=True, gamma="auto") )),
# ])





# LabelSpreading Pipeline
ls_pipeline = Pipeline([
    ('vect', CountVectorizer(**vectorizer_params)),
    ('tfidf', TfidfTransformer()),
    # LabelSpreading does not support dense matrices
    ('todense', FunctionTransformer(lambda x: x.todense())),
    ('clf', LabelSpreading()),
])


def eval_and_print_metrics(clf, X_train, y_train, X_test, y_test):
    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # if -1 in y_train:
    #   print("Y-PRED-PROBA", clf.predict_proba(X_train))

    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))
    print("Classification Report: \n", classification_report(y_test, y_pred))
    print("-" * 10)
    print()



def eval_and_print_metrics_df(clf, X_train, y_train, X_test, y_test, thresh = None, kbest = None):

    dict1 = {}


    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    
    dict1['Labeled'] = len(X_train) - sum(1 for x in y_train if x == -1)
    dict1['UnLabeled'] = sum(1 for x in y_train if x == -1)


    
    # if sum(1 for x in y_train if x == -1) == 0:
    #     dict1['type'] = 'Supervised'
    # else:
    #     dict1['type'] = 'Semi-Supervised'

    dict1['Threshold'] = thresh
    dict1['K-Best'] = kbest

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))

    dict1['Accuracy'] = accuracy_score(y_test, y_pred)

    print("-" * 10)
    print()

    return dict1

In [ ]:
import pandas as pd
if __name__ == "__main__":

    # n_list = [10, 200, 500]
    n_list = [10, 50, 200, 500, 1000]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 1000

      data_path = './data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised SGDClassifier on ", 2*n," of the training data:")
      eval_and_print_metrics(pipeline, X_20, y_20, X_test, y_test)

      # set the non-masked subset to be unlabeled
      # y_train[~y_mask] = -1
      print("SelfTrainingClassifier on ", 2*n," of the training data (rest "
          "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
        # SelfTraining Pipeline
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(SGDClassifier(**sdg_params), threshold = t, verbose=True)),
        ])

        eval_and_print_metrics(st_pipeline, X_train, y_train, X_test, y_test)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


FileNotFoundError: ignored

In [ ]:
import pandas as pd
if __name__ == "__main__":

    # n_list = [10, 200, 500]
    n_list = [10, 50, 200, 500, 1000]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 2500

      data_path = './data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised SGDClassifier on ", 2*n," of the training data:")
      eval_and_print_metrics(pipeline, X_20, y_20, X_test, y_test)

      # set the non-masked subset to be unlabeled
      # y_train[~y_mask] = -1
      print("SelfTrainingClassifier on ", 2*n," of the training data (rest "
          "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
        # SelfTraining Pipeline
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(SGDClassifier(**sdg_params), threshold = t, verbose=True)),
        ])

        eval_and_print_metrics(st_pipeline, X_train, y_train, X_test, y_test)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


KeyboardInterrupt: ignored

In [ ]:
# Naive Bayes for IMDB
from sklearn.naive_bayes import MultinomialNB

# Parameters
# mnb_params = 
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_nb_imdb_2k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 1000

      data_path = './data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised NBClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', MultinomialNB()),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_nb_imdb_2k = df_nb_imdb_2k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for kb in kbest_list:
        kbest = int((unlabeled_per_class*2)/kb)
        print("---------------------------------K-Best = ", kbest,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(MultinomialNB(), criterion = 'k_best', k_best = kbest, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = None, kbest = kbest)
        df_nb_imdb_2k = df_nb_imdb_2k.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised NBClassifier on 10% of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.590
Accuracy Score:  0.59016
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------K-Best =  500 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 500 new labels.
End of iteration 2, added 500 new labels.
End of iteration 3, added 500 new labels.
End of iteration 4, added 500 new labels.
Micro-averaged F1 score on test set: 0.493
Accuracy Score:  0.49256
----------

---------------------------------K-Best =  400 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 400 new labels.
End of iteration 2, added 400 new

In [ ]:
display(df_nb_imdb_2k[['Labeled', 'UnLabeled', 'Threshold', 'K-Best', 'Accuracy']].sort_values(['Labeled', 'UnLabeled']))

In [ ]:
df_nb_imdb_2k.pivot(index=['Labeled', 'UnLabeled'], columns='K-Best')['Accuracy']

K-Best               NaN      250.0    285.0    333.0    400.0    500.0
Labeled UnLabeled                                                      
20.0    0.0        0.59016      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
100.0   0.0        0.68456      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
400.0   0.0        0.79504      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
1000.0  0.0        0.83752      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
2000.0  0.0        0.85776      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.49256  0.49256  0.49256  0.49280  0.49504

In [ ]:
# Naive Bayes for IMDB
from sklearn.naive_bayes import MultinomialNB

# Parameters
# mnb_params = 
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_nb_imdb_5k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 2500

      data_path = './data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised NBClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', MultinomialNB()),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_nb_imdb_5k = df_nb_imdb_5k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for kb in kbest_list:
        kbest = int((unlabeled_per_class*2)/kb)
        print("---------------------------------K-Best = ", kbest,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(MultinomialNB(), criterion = 'k_best', k_best = kbest, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = None, kbest = kbest)
        df_nb_imdb_5k = df_nb_imdb_5k.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised NBClassifier on 10% of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.590
Accuracy Score:  0.59016
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------K-Best =  1250 ---------------------------------
Number of training samples: 5020
Unlabeled samples in training set: 5000
End of iteration 1, added 1250 new labels.
End of iteration 2, added 1250 new labels.
End of iteration 3, added 1250 new labels.
End of iteration 4, added 1250 new labels.
Micro-averaged F1 score on test set: 0.493
Accuracy Score:  0.49256
----------

---------------------------------K-Best =  1000 ---------------------------------
Number of training samples: 5020
Unlabeled samples in training set: 5000
End of iteration 1, added 1000 new labels.
End of iteration 2, added 

In [ ]:
display(df_nb_imdb_5k[['Labeled', 'UnLabeled', 'Threshold', 'K-Best', 'Accuracy']].sort_values(['Labeled', 'UnLabeled']))

,Labeled,UnLabeled,Threshold,K-Best,Accuracy
0,20.0,0.0,NaN,NaN,0.59016
1,20.0,5000.0,NaN,1250.0,0.49256
2,20.0,5000.0,NaN,1000.0,0.49256
3,20.0,5000.0,NaN,833.0,0.49256
4,20.0,5000.0,NaN,714.0,0.49256
5,20.0,5000.0,NaN,625.0,0.49256
6,100.0,0.0,NaN,NaN,0.68456
7,100.0,5000.0,NaN,1250.0,0.49256
8,100.0,5000.0,NaN,1000.0,0.49256
9,100.0,5000.0,NaN,833.0,0.49256


In [ ]:
df_nb_imdb_5k.pivot(index=['Labeled', 'UnLabeled'], columns='K-Best')['Accuracy']

K-Best              NaN      625.0    714.0    833.0    1000.0   1250.0
Labeled UnLabeled                                                      
20.0    0.0        0.59016      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
100.0   0.0        0.68456      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
400.0   0.0        0.79504      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
1000.0  0.0        0.83752      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256
2000.0  0.0        0.85776      NaN      NaN      NaN      NaN      NaN
        5000.0         NaN  0.49256  0.49256  0.49256  0.49256  0.49256

In [ ]:
# Random Forest for IMDB
from sklearn.ensemble import RandomForestClassifier

# Parameters
# mnb_params = 
rf_params = dict(n_estimators=1000, random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_rf_imdb_2k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 1000

      data_path = './data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised NBClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', RandomForestClassifier(**rf_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_rf_imdb_2k = df_rf_imdb_2k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(RandomForestClassifier(**rf_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = t, kbest = None)
        df_rf_imdb_2k = df_rf_imdb_2k.append(temp, ignore_index=True)

    


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 20~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Supervised NBClassifier on 10% of the training data:
Number of training samples: 20
Unlabeled samples in training set: 0
Micro-averaged F1 score on test set: 0.621
Accuracy Score:  0.6208
----------

SelfTrainingClassifier on 10% of the training data (rest is unlabeled):
---------------------------------Threshold =  0.4 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 2000 new labels.
Micro-averaged F1 score on test set: 0.508
Accuracy Score:  0.50824
----------

---------------------------------Threshold =  0.5 ---------------------------------
Number of training samples: 2020
Unlabeled samples in training set: 2000
End of iteration 1, added 1990 new labels.
End of iteration 2, added 10 new labels.
Micro-averaged F1 score on test set: 0.508
Accuracy Score:  0.50816
----------

-------------------------------

In [ ]:
df_rf_imdb_2k.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

Threshold              NaN      0.4      0.5      0.6      0.7      0.8
Labeled UnLabeled                                                      
20.0    0.0        0.62080      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.50824  0.50816  0.50744  0.50744  0.58864
100.0   0.0        0.63768      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.57832  0.57312  0.51256  0.50744  0.64296
400.0   0.0        0.77016      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.76912  0.76520  0.58488  0.55544  0.78024
1000.0  0.0        0.81184      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.80160  0.79896  0.75048  0.62816  0.80808
2000.0  0.0        0.83816      NaN      NaN      NaN      NaN      NaN
        2000.0         NaN  0.82768  0.82504  0.79760  0.79680  0.83280

In [ ]:
# SVM for IMDB
from sklearn.linear_model import SGDClassifier

# Parameters
# mnb_params = 
svm_params = dict(alpha=1e-5, penalty='l2', loss='hinge', random_state=0)
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)


if __name__ == "__main__":

    df_svm_imdb_2k = pd.DataFrame()

    n_list = [10, 50, 200, 500, 1000]
    kbest_list=[4, 5, 6, 7, 8]
    threshold=[0.4, 0.5, 0.6, 0.7, 0.8]

for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 1000

      data_path = './data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      X_20, y_20 = texts[:2*n], labels[:2*n]
      print("Supervised NBClassifier on "+str(n)+"% of the training data:")

      # Supervised Pipeline
      pipeline = Pipeline([
          ('vect', CountVectorizer(**vectorizer_params)),
          ('tfidf', TfidfTransformer()),
          ('clf', SGDClassifier(**svm_params)),
      ])

      
      temp = eval_and_print_metrics_df(pipeline, X_20, y_20, X_test, y_test, thresh = None, kbest = None)
      df_svm_imdb_2k = df_svm_imdb_2k.append(temp, ignore_index=True)

      # set the non-masked subset to be unlabeled
      # set only 50% of data to be unlabeled in every iteration of training.
      print("SelfTrainingClassifier on "+str(n)+"% of the training data (rest "
            "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
      # X_50, y_50 = map(list, zip(*((x, y)
      #                 for x, y, m in zip(X_train, y_train, y_mask) if m)))
        # SelfTraining Pipeline
        
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(SGDClassifier(**svm_params), criterion = 'threshold', threshold = t, verbose=True)),
        ])
        temp = eval_and_print_metrics_df(st_pipeline, X_train, y_train, X_test, y_test, thresh = t, kbest = None)
        df_svm_imdb_2k = df_svm_imdb_2k.append(temp, ignore_index=True)

    


In [ ]:
df_svm_imdb_2k.pivot(index=['Labeled', 'UnLabeled'], columns='Threshold')['Accuracy']

In [ ]:
#Checking if threshold parameter for SelfTrainingClassifier is used for adding examples selectively or as a threshold to predict_proba.
#If it is used for predict_proba, then we should compare it with the supervised version with the same threshold - not with supervised(threshold=0.5).

def eval_and_print_metrics_2(clf, X_train, y_train, X_test, y_test):
    print("Number of training samples:", len(X_train))
    print("Unlabeled samples in training set:",
          sum(1 for x in y_train if x == -1))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_pred_proba = clf.predict_proba(X_test)
    print("Micro-averaged F1 score on test set: "
          "%0.3f" % f1_score(y_test, y_pred, average='micro'))
    print("Accuracy Score: ", accuracy_score(y_test, y_pred))
    print("-" * 10)
    print()
    return y_pred, y_pred_proba



import pandas as pd
if __name__ == "__main__":

    # n_list = [10, 200, 500]
    n_list = [10, 50, 200, 500, 1000]
    n_list = [200]
    threshold = [0.4, 0.5, 0.6, 0.7, 0.8]
    threshold = [0.8]

    for n in n_list:
      print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = "+str(2*n)+"~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

      

      n_labeled_per_class = n       #10, 200, 500, 1000, 2400
      unlabeled_per_class = 2500

      data_path = './data/imdb_data/'
      train_df = pd.read_csv(data_path+'train.csv', header=None)
      test_df = pd.read_csv(data_path+'test.csv', header=None)

      train_labels = np.array([v for v in train_df[1]])
      train_text = np.array([v for v in train_df[0]])
      test_labels = np.array([u for u in test_df[1]])
      test_text = np.array([v for v in test_df[0]])

      n_labels = 2
      # Split the labeled training set, unlabeled training set, development set
      train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
          train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

      # print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      #     train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

      df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
      # print(df_train.shape)
      # df_train.head()

      df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
      # print(df_test.shape)



      df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
      # print(df_unlabeled.shape)
      # df_unlabeled.head()

      clean_train_df = clean(df_train)
      clean_test_df = clean(df_test)
      clean_unlabeled_df = clean(df_unlabeled)

      texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))


      labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

      X_test = np.array(clean_test_df.review)
      y_test = np.array(clean_test_df.sentiment)

      X_train = texts
      y_train = labels


      # X, y = data.data, data.target
      # X_train, X_test, y_train, y_test = train_test_split(X, y)

      # print("Supervised SGDClassifier on 100% of the data:")
      # eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

      np.random.seed(0)

      # X_20 and y_20 are the subset of the train dataset indicated by the mask
      # X_20, y_20 = texts[:2*n], labels[:2*n]
      # print("Supervised SGDClassifier on ", 2*n," of the training data:")
      # eval_and_print_metrics(pipeline, X_20, y_20, X_test, y_test)

      # set the non-masked subset to be unlabeled
      # y_train[~y_mask] = -1
      print("SelfTrainingClassifier on ", 2*n," of the training data (rest "
          "is unlabeled):")
      for t in threshold:
        print("---------------------------------Threshold = ", t,"---------------------------------")
      
        # SelfTraining Pipeline
        st_pipeline = Pipeline([
            ('vect', CountVectorizer(**vectorizer_params)),
            ('tfidf', TfidfTransformer()),
            ('clf', SelfTrainingClassifier(SGDClassifier(**sdg_params), threshold = t, verbose=True)),
        ])

        y_pred, y_proba = eval_and_print_metrics_2(st_pipeline, X_train, y_train, X_test, y_test)

        print("t = ",t, '\n', y_pred)
        print("t = ",t, '\n', y_proba)

        print("Number of equal values between y_pred and y_pred(calculated) when we calculate y_pred from y_proba with threshold = 0.8: ", sum(np.equal((y_proba[:,1] >= 0.8).astype(int), y_pred)))
        print("Number of equal values between y_pred and y_pred(calculated) when we calculate y_pred from y_proba with threshold = 0.5: ", sum(np.equal((y_proba[:,1] >= 0.5).astype(int), y_pred)))


#Conclusion: Threshold is used as a part of training but not to calculate pred from pred_proba.



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~IMDB DATA with n_labeled = 400~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
SelfTrainingClassifier on  400  of the training data (rest is unlabeled):
---------------------------------Threshold =  0.8 ---------------------------------
Number of training samples: 5400
Unlabeled samples in training set: 5000
End of iteration 1, added 2376 new labels.
End of iteration 2, added 1084 new labels.
End of iteration 3, added 280 new labels.
End of iteration 4, added 95 new labels.
End of iteration 5, added 45 new labels.
End of iteration 6, added 23 new labels.
End of iteration 7, added 8 new labels.
End of iteration 8, added 6 new labels.
End of iteration 9, added 9 new labels.
End of iteration 10, added 3 new labels.
Micro-averaged F1 score on test set: 0.805
Accuracy Score:  0.8052
----------

t =  0.8 
 [0 0 0 ... 0 1 1]
t =  0.8 
 [[0.89715549 0.10284451]
 [0.57326692 0.42673308]
 [0.93212653 0.06787347]
 ...
 [0.74134373 0.25865627]
 [0.00151803 0.99848197]
 [0.0892771

In [ ]:
def train_val_split(labels, n_labeled_per_class, unlabeled_per_class, n_labels, seed=0):
    """Split the original training set into labeled training set, unlabeled training set, development set

    Arguments:
        labels {list} -- List of labeles for original training set
        n_labeled_per_class {int} -- Number of labeled data per class
        unlabeled_per_class {int} -- Number of unlabeled data per class
        n_labels {int} -- The number of classes

    Keyword Arguments:
        seed {int} -- [random seed of np.shuffle] (default: {0})

    Returns:
        [list] -- idx for labeled training set, unlabeled training set, development set
    """
    np.random.seed(seed)
    labels = np.array(labels)
    train_labeled_idxs = []
    train_unlabeled_idxs = []
    val_idxs = []

    for i in range(n_labels):
        idxs = np.where(labels == i)[0]
        np.random.shuffle(idxs)
        if n_labels == 2:
            # IMDB
            
            
            
            n_unlabeled_per_class = 1000   #10, 100, 500, 1000, 2500
            train_pool = np.concatenate((idxs[:500], idxs[5500:-2000]))
            train_labeled_idxs.extend(train_pool[:n_labeled_per_class])
            train_unlabeled_idxs.extend(idxs[500: 500 + n_unlabeled_per_class])
            val_idxs.extend(idxs[-2000:])
        
        
        
            # train_pool = np.concatenate((idxs[:500], idxs[5500:-2000]))
            # train_labeled_idxs.extend(train_pool[:n_labeled_per_class])
            # train_unlabeled_idxs.extend(
            #     idxs[500: 500 + 5000])
            # val_idxs.extend(idxs[-2000:])
            

    return train_labeled_idxs, train_unlabeled_idxs, val_idxs



In [ ]:
%%time
import re
#Removes Punctuations
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

#Removes Emojis
def remove_emoji(data):
    emoji_clean= re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    data=emoji_clean.sub(r'',data)
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data


CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 13.4 µs


In [ ]:
def clean(train_df):
    
    train_df['review']=train_df['review'].apply(lambda z: remove_punctuations(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_html(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_url(z))
    train_df['review']=train_df['review'].apply(lambda z: remove_emoji(z))
    
    train_df['review']=train_df['review'].apply(lambda z: word_tokenize(z))
    
    lemmatizer = WordNetLemmatizer()
#     train_df['review']=train_df['review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    train_df['review']=train_df['review'].apply(lambda x: ' '.join(x))
    
    return train_df

In [ ]:
def make_feature_vec(words, model, num_features):
    """
    Average the word vectors for a set of words
    """
    feature_vec = np.zeros((num_features,),dtype="float32")  # pre-initialize (for speed)
    nwords = 0.
    index2word_set = set(model.wv.index2word)  # words known to the model

    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            feature_vec = np.add(feature_vec,model[word])
    
    feature_vec = np.divide(feature_vec, nwords)
    return feature_vec


def get_avg_feature_vecs(reviews, model, num_features):
    """
    Calculate average feature vectors for all reviews
    """
    counter = 0
    review_feature_vecs = np.zeros((len(reviews),num_features), dtype='float32')  # pre-initialize (for speed)
    
    for review in reviews:
#         print(counter)
        review_feature_vecs[counter] = make_feature_vec(review, model, num_features)
        counter = counter + 1
    return review_feature_vecs


In [ ]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import word2vec
from gensim.models import KeyedVectors


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def get_data_tfidf(n, arg1=1):

  n_labeled_per_class = n       #10, 200, 500, 1000, 2400
  unlabeled_per_class = 1000


  data_path = './data/imdb_data/'
  train_df = pd.read_csv(data_path+'train.csv', header=None)
  test_df = pd.read_csv(data_path+'test.csv', header=None)
  print("Read data successful", train_df.shape, test_df.shape, "n_labeled = ", n_labeled_per_class*2)
  print(train_df.head(5))


  train_labels = np.array([v for v in train_df[1]])
  train_text = np.array([v for v in train_df[0]])
  print(train_labels[:3])
  print(train_text[:3])

  test_labels = np.array([u for u in test_df[1]])
  test_text = np.array([v for v in test_df[0]])




  n_labels = 2
  # Split the labeled training set, unlabeled training set, development set
  train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
      train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

  print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))
  


  df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
  print(df_train.shape)
  # df_train.head()

  df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
  print(df_test.shape)

  if arg1!=1:
    return df_test
  # df_test.head()

  df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
  print(df_unlabeled.shape)
  # df_unlabeled.head()

  clean_train_df = clean(df_train)
  clean_test_df = clean(df_test)
  clean_unlabeled_df = clean(df_unlabeled)

  texts = np.array((clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)))

  # Create Corpus
  # model = word2vec.Word2Vec(texts, min_count=1)

  NGRAM_RANGE = (1, 2)
  TOP_K = 20000
  TOKEN_MODE = 'word'
  MIN_DOC_FREQ = 2

  kwargs = {
      'ngram_range' : NGRAM_RANGE,
      'dtype' : 'int32',
      'strip_accents' : 'unicode',
      'decode_error' : 'replace',
      'analyzer' : TOKEN_MODE,
      'min_df' : MIN_DOC_FREQ,
  }

  tfidf_vectorizer = TfidfVectorizer(**kwargs)


  ##save the modeled words produced from Word2Vec
  # model.save('word2vec_model.bin')


  # loaded_model=KeyedVectors.load('word2vec_model.bin')
  # print("Loaded word2vec model:", loaded_model)

  # num_features = 100 # See Word2Vec(vocab=213802, size=100, alpha=0.025) in above cell.
  # feature_vec_train = get_avg_feature_vecs(clean_train_df['review'], loaded_model, num_features)
  # feature_vec_test = get_avg_feature_vecs(clean_test_df['review'], loaded_model, num_features)
  # feature_vec_unlabeled = get_avg_feature_vecs(clean_unlabeled_df['review'], loaded_model, num_features)

  feature_vec_full_train = tfidf_vectorizer.fit_transform(texts)

  feature_vec_train = tfidf_vectorizer.transform(clean_train_df['review'])
  feature_vec_test = tfidf_vectorizer.transform(clean_test_df['review'])
  feature_vec_unlabeled = tfidf_vectorizer.transform(clean_unlabeled_df['review'])


  TOP_K = 20000
  selector = SelectKBest(f_classif, k=min(TOP_K, feature_vec_train.shape[1]))

  selector.fit(feature_vec_train, np.array([i for i in list(df_train.sentiment)]))

  feature_vec_train = selector.transform(feature_vec_train).astype('float32').toarray()
  feature_vec_test = selector.transform(feature_vec_test).astype('float32').toarray()
  feature_vec_unlabeled = selector.transform(feature_vec_unlabeled).astype('float32').toarray()


  print("TYPE", type(feature_vec_train))
  feature_vec_full_train = np.concatenate((feature_vec_train, feature_vec_unlabeled))
  print("Shape of train data: ",feature_vec_full_train.shape)

  labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

  return feature_vec_full_train, feature_vec_test, labels, test_labels, df_test

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!ls

 code	     LICENSE			'Running MixText v5 ISIS Data.ipynb'
 data	     README.md			'Semi-Supervised sklearn.ipynb'
 data_temp  'Running MixText v3.ipynb'	 word2vec_model.bin


In [ ]:
# For tf-idf

n_list = [10, 200, 500, 1000, 2400]
# n_list=[10]

for i in n_list:

  train_vec, test_vec, labels, test_labels, df_test = get_data_tfidf(i)

  print("TRAIN VEC: ", train_vec.shape)
  np.save('data_temp/tf-idf/train_vec_'+str(i), train_vec)
  np.save('data_temp/tf-idf/test_vec_'+str(i), test_vec)
  np.save('data_temp/tf-idf/labels_'+str(i), labels)
  np.save('data_temp/tf-idf/test_labels_'+str(i), test_labels)

  print(i, 'SAVED!!')

Read data successful (37500, 2) (12500, 2) n_labeled =  20
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not deliver enough laughs, twists, or tension to k

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1804: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


TYPE <class 'numpy.ndarray'>
Shape of train data:  (2020, 20000)
TRAIN VEC:  (2020, 20000)
10 SAVED!!
Read data successful (37500, 2) (12500, 2) n_labeled =  400
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin becaus

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1804: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


TYPE <class 'numpy.ndarray'>
Shape of train data:  (2400, 20000)
TRAIN VEC:  (2400, 20000)
200 SAVED!!
Read data successful (37500, 2) (12500, 2) n_labeled =  1000
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin beca

...

TYPE <class 'numpy.ndarray'>
Shape of train data:  (3000, 20000)
TRAIN VEC:  (3000, 20000)
500 SAVED!!
Read data successful (37500, 2) (12500, 2) n_labeled =  2000
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin beca

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1804: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


TYPE <class 'numpy.ndarray'>
Shape of train data:  (4000, 20000)
TRAIN VEC:  (4000, 20000)
1000 SAVED!!
Read data successful (37500, 2) (12500, 2) n_labeled =  4800
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin bec

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:1804: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


TYPE <class 'numpy.ndarray'>
Shape of train data:  (6800, 20000)
TRAIN VEC:  (6800, 20000)
2400 SAVED!!


In [ ]:
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import accuracy_score

n_list = [10, 200, 500, 1000, 2400]

acc_list = {}

for i in n_list:

  df_test = get_data_tfidf(i, 10)

  train_vec = np.load('data_temp/tf-idf/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/tf-idf/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/tf-idf/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')

  print("HELLO!", train_vec.shape)

  label_prop_model = LabelSpreading()
  label_prop_model.fit(train_vec, labels)

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = label_prop_model.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

Read data successful (37500, 2) (12500, 2) n_labeled =  20
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not deliver enough laughs, twists, or tension to k

In [ ]:
train_vec = np.load('data_temp/tf-idf/train_vec_'+str(10)+'.npy')
train_vec

array([[0.04196495, 0.07739466, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02940054, 0.        , 0.05313623, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [ ]:
print('---------Label Spreading---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list[i])

---------Label Spreading---------
Accuracy for  20  labeled data is: 54.040
Accuracy for  400  labeled data is: 67.456
Accuracy for  1000  labeled data is: 68.272
Accuracy for  2000  labeled data is: 52.896
Accuracy for  4800  labeled data is: 51.088


In [ ]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score

n_list = [10, 200, 500, 1000, 2400]

acc_list = {}

for i in n_list:

  df_test = get_data_tfidf(i, 10)

  train_vec = np.load('data_temp/tf-idf/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/tf-idf/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/tf-idf/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')


  label_prop_model = LabelPropagation()
  label_prop_model.fit(train_vec, labels)

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = label_prop_model.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

Read data successful (37500, 2) (12500, 2) n_labeled =  20
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not deliver enough laughs, twists, or tension to k

/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:292: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


Accuracy for  20  labeled data is: 50.824
Read data successful (37500, 2) (12500, 2) n_labeled =  400
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not del

/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:292: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


Accuracy for  400  labeled data is: 49.272
Read data successful (37500, 2) (12500, 2) n_labeled =  1000
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not d

/usr/local/lib/python3.7/dist-packages/sklearn/semi_supervised/_label_propagation.py:292: ConvergenceWarning: max_iter=1000 was reached without convergence.
  category=ConvergenceWarning


Accuracy for  1000  labeled data is: 49.592
Read data successful (37500, 2) (12500, 2) n_labeled =  2000
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not 

In [ ]:
print('---------Label Propagation---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list[i])

---------Label Propagation---------
Accuracy for  20  labeled data is: 50.824
Accuracy for  400  labeled data is: 49.272
Accuracy for  1000  labeled data is: 49.592
Accuracy for  2000  labeled data is: 78.264
Accuracy for  4800  labeled data is: 52.776


In [ ]:
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

n_list = [10, 200, 500, 1000, 2400]

acc_list = {}

for i in n_list:

  df_test = get_data_tfidf(i, 10)

  train_vec = np.load('data_temp/tf-idf/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/tf-idf/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/tf-idf/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')

  
  svc = SVC(probability=True, gamma="auto") 


  label_prop_model = SelfTrainingClassifier(svc)
  label_prop_model.fit(train_vec, labels)

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = label_prop_model.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

Read data successful (37500, 2) (12500, 2) n_labeled =  20
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not deliver enough laughs, twists, or tension to k

In [ ]:
print('---------Self Training Classifier with SVM---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list[i])

---------Self Training Classifier with SVM---------
Accuracy for  20  labeled data is: 63.672
Accuracy for  400  labeled data is: 76.496
Accuracy for  1000  labeled data is: 76.712
Accuracy for  2000  labeled data is: 80.968
Accuracy for  4800  labeled data is: 79.168


In [ ]:
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#Without Unlabeled Data

n_list = [10, 200, 500, 1000, 2400]
n_list = [10, 200, 500, 1000]
acc_list_2 = {}

for i in n_list:

  df_test = get_data_tfidf(i, 10)

  train_vec = np.load('data_temp/tf-idf/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/tf-idf/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/tf-idf/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')

  
  svc = SVC(probability=True, gamma="auto") 

  print("SHAPE: ", train_vec.shape, "SIZE: ", labels.shape)
  print("New SHAPE: ", train_vec[:2*i].shape, "New SIZE: ", labels[:2*i].shape, np.unique(labels), np.unique(labels[:2*i]))

  svc.fit(train_vec[:2*i], labels[:2*i])

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = svc.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list_2[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

Read data successful (37500, 2) (12500, 2) n_labeled =  20
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1
[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not deliver enough laughs, twists, or tension to k

In [ ]:
print('---------Label Spreading---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list_2[i])

---------Label Spreading---------
Accuracy for  20  labeled data is: 63.672
Accuracy for  400  labeled data is: 76.496
Accuracy for  1000  labeled data is: 76.712
Accuracy for  2000  labeled data is: 80.968


In [ ]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import word2vec
from gensim.models import KeyedVectors

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def get_data(n, arg1=1):

  n_labeled_per_class = n       #10, 200, 500, 1000, 2400
  unlabeled_per_class = 1000


  data_path = './data/imdb_data/'
  train_df = pd.read_csv(data_path+'train.csv', header=None)
  test_df = pd.read_csv(data_path+'test.csv', header=None)
  print("Read data successful", train_df.shape, test_df.shape, "n_labeled = ", n_labeled_per_class*2)
  print(train_df.head(5))


  train_labels = np.array([v for v in train_df[1]])
  train_text = np.array([v for v in train_df[0]])
  print(train_labels[:3])
  print(train_text[:3])

  test_labels = np.array([u for u in test_df[1]])
  test_text = np.array([v for v in test_df[0]])




  n_labels = 2
  # Split the labeled training set, unlabeled training set, development set
  train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
      train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

  print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
      train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))
  


  df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
  print(df_train.shape)
  # df_train.head()

  df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
  print(df_test.shape)

  if arg1!=1:
    return df_test
  # df_test.head()

  df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
  print(df_unlabeled.shape)
  # df_unlabeled.head()

  clean_train_df = clean(df_train)
  clean_test_df = clean(df_test)
  clean_unlabeled_df = clean(df_unlabeled)

  texts = (clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)).apply(lambda x:x.split())

  # Create Corpus
  embedding_vector_size = 300
  model = word2vec.Word2Vec(sentences = texts, size = embedding_vector_size, min_count=3, window=5, workers=4)

  ##save the modeled words produced from Word2Vec
  model.save('word2vec_model.bin')


  loaded_model=KeyedVectors.load('word2vec_model.bin')
  print("Loaded word2vec model:", loaded_model)

  num_features = 300 # See Word2Vec(vocab=213802, size=100, alpha=0.025) in above cell.
  feature_vec_train = get_avg_feature_vecs(clean_train_df['review'], loaded_model, num_features)
  feature_vec_test = get_avg_feature_vecs(clean_test_df['review'], loaded_model, num_features)
  feature_vec_unlabeled = get_avg_feature_vecs(clean_unlabeled_df['review'], loaded_model, num_features)



  feature_vec_full_train = np.concatenate((feature_vec_train, feature_vec_unlabeled))
  print("Shape of train data: ",feature_vec_full_train.shape)

  labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])

  return feature_vec_full_train, feature_vec_test, labels, test_labels, df_test


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# For word2vec

n_list = [10, 200, 500, 1000, 2400]

for i in n_list:

  train_vec, test_vec, labels, test_labels, df_test = get_data(i)

  np.save('data_temp/train_vec_'+str(i), train_vec)
  np.save('data_temp/test_vec_'+str(i), test_vec)
  np.save('data_temp/labels_'+str(i), labels)
  np.save('data_temp/test_labels_'+str(i), test_labels)

  print(i, 'SAVED!!')

SyntaxError: ignored

In [ ]:
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import accuracy_score

n_list = [10, 200, 500, 1000, 2400]

acc_list = {}

for i in n_list:

  df_test = get_data(i, 10)

  train_vec = np.load('data_temp/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')


  label_prop_model = LabelSpreading()
  label_prop_model.fit(train_vec, labels)

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = label_prop_model.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

In [ ]:
print('---------Label Spreading---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list[i])

---------Label Spreading---------
Accuracy for  20  labeled data is: 49.584
Accuracy for  400  labeled data is: 53.720
Accuracy for  1000  labeled data is: 54.448
Accuracy for  2000  labeled data is: 54.480
Accuracy for  4800  labeled data is: 54.536


In [ ]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score

n_list = [10, 200, 500, 1000, 2400]

acc_list = {}

for i in n_list:

  df_test = get_data(i, 10)

  train_vec = np.load('data_temp/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')


  label_prop_model = LabelPropagation()
  label_prop_model.fit(train_vec, labels)

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = label_prop_model.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

In [ ]:
print('---------Label Propagation---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list[i])

---------Label Propagation---------
Accuracy for  20  labeled data is: 49.256
Accuracy for  400  labeled data is: 51.400
Accuracy for  1000  labeled data is: 54.152
Accuracy for  2000  labeled data is: 54.272
Accuracy for  4800  labeled data is: 54.400


In [ ]:
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

n_list = [10, 200, 500, 1000, 2400]

acc_list = {}

for i in n_list:

  df_test = get_data(i, 10)

  train_vec = np.load('data_temp/train_vec_'+str(i)+'.npy')
  test_vec = np.load('data_temp/test_vec_'+str(i)+'.npy')
  labels = np.load('data_temp/labels_'+str(i)+'.npy')
  # test_labels = np.load('data_temp/test_labels_'+str(i)+'.npy')

  
  svc = SVC(probability=True, gamma="auto") 


  label_prop_model = SelfTrainingClassifier(svc)
  label_prop_model.fit(train_vec, labels)

  test_labels = np.array([i for i in list(df_test.sentiment)])
  y_hat = label_prop_model.predict(test_vec)
  score = accuracy_score(test_labels, y_hat)

  acc_list[i] = score*100
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % (score*100))

In [ ]:
print('---------Self Training Classifier with SVM---------')
for i in n_list:
  print('Accuracy for ', i*2,  ' labeled data is: %.3f' % acc_list[i])

---------Self Training Classifier with SVM---------
Accuracy for  20  labeled data is: 52.928
Accuracy for  400  labeled data is: 53.400
Accuracy for  1000  labeled data is: 54.120
Accuracy for  2000  labeled data is: 54.944
Accuracy for  4800  labeled data is: 55.224


In [ ]:
import sklearn
sklearn.show_versions()


System:
    python: 3.7.10 (default, May  3 2021, 02:48:31)  [GCC 7.5.0]
executable: /usr/bin/python3
   machine: Linux-5.4.109+-x86_64-with-Ubuntu-18.04-bionic

Python dependencies:
          pip: 19.3.1
   setuptools: 56.1.0
      sklearn: 0.24.2
        numpy: 1.19.5
        scipy: 1.4.1
       Cython: 0.29.23
       pandas: 1.1.5
   matplotlib: 3.2.2
       joblib: 1.0.1
threadpoolctl: 2.1.0

Built with OpenMP: True


In [ ]:
n_labeled_per_class = 200       #10, 200, 500, 1000, 2400
unlabeled_per_class = 1000

In [ ]:
data_path = './data/imdb_data/'
train_df = pd.read_csv(data_path+'train.csv', header=None)
test_df = pd.read_csv(data_path+'test.csv', header=None)
print("Read data successful", train_df.shape, test_df.shape, "n_labeled = ", n_labeled_per_class)
print(train_df.head(5))

Read data successful (37500, 2) (12500, 2) n_labeled =  200
                                                   0  1
0  I figure this to be an "alternate reality" tee...  0
1  This is the kind of movie that wants to be goo...  0
2  This was by far the worst movie I've ever seen...  0
3  Awful, awful, awful...<br /><br />I loved the ...  0
4  Fragile Carne, just before his great period. A...  1


In [ ]:
train_labels = np.array([v for v in train_df[1]])
train_text = np.array([v for v in train_df[0]])
print(train_labels[:3])
print(train_text[:3])

test_labels = np.array([u for u in test_df[1]])
test_text = np.array([v for v in test_df[0]])

[0 0 0]
['I figure this to be an "alternate reality" teen flick...More precisely a Ferris Bueller type character as the leader of a cheat ring . Yeah, I know it\'s not meant to compared to Ferris Bueller, at least not in a "oranges-to-oranges" way, but it will none-the-less.<br /><br />Bottom-line: It\'s galaxies away from even being even a minor classic. It is watchable, though only if you\'re not expecting very much. That said, the main character has some charm, but the premise wears thin because the writing just isn\'t clever. The movie just did not deliver enough laughs, twists, or tension to keep my interest. <br /><br />To be honest I did continue watching...Watching with hopes to see if anything suddenly clicked. It didn\'t. So, stylish as it is, I wouldn\'t recommend this movie. BTW, it seems odd to see Mary Tyler Moore as the principal. She\'s truly miscast, I hope the paycheck was inordinately big.'
 "This is the kind of movie that wants to be good but sucks. First thing, wha

In [ ]:
n_labels = 2
# Split the labeled training set, unlabeled training set, development set
train_labeled_idxs, train_unlabeled_idxs, val_idxs = train_val_split(
    train_labels, n_labeled_per_class, unlabeled_per_class, n_labels)

# Build the dataset class for each set
# train_labeled_dataset = loader_labeled(
#     train_text[train_labeled_idxs], train_labels[train_labeled_idxs], tokenizer, max_seq_len, train_aug)
# train_unlabeled_dataset = loader_unlabeled(
#     train_text[train_unlabeled_idxs], train_unlabeled_idxs, tokenizer, max_seq_len, Translator(data_path))
# val_dataset = loader_labeled(
#     train_text[val_idxs], train_labels[val_idxs], tokenizer, max_seq_len)
# test_dataset = loader_labeled(
#     test_text, test_labels, tokenizer, max_seq_len)

print("#Labeled: {}, Unlabeled {}, Val {}, Test {}".format(len(
    train_labeled_idxs), len(train_unlabeled_idxs), len(val_idxs), len(test_labels)))

#Labeled: 400, Unlabeled 2000, Val 4000, Test 12500


In [ ]:
df_train = pd.DataFrame({'review':train_text[train_labeled_idxs], 'sentiment':train_labels[train_labeled_idxs]})
print(df_train.shape)
df_train.head()

(400, 2)


,review,sentiment
0,"okay, let's cut to the chase - there's no way ...",0
1,I saw Hurlyburly on Broadway and liked it a gr...,0
2,"First of all, I'd like to tell you that I'm in...",0
3,Harlan Knowles (Lance Henriksen) brings a grou...,0
4,Ugh. Pretty awful.<br /><br />Linnea Quigley g...,0


In [ ]:
df_test = pd.DataFrame({'review':test_text, 'sentiment':test_labels})
print(df_test.shape)
df_test.head()

(12500, 2)


,review,sentiment
0,I really liked this Summerslam due to the look...,1
1,Not many television shows appeal to quite as m...,1
2,The film quickly gets to a major chase scene w...,0
3,Jane Austen would definitely approve of this o...,1
4,Expectations were somewhat high for me when I ...,0


In [ ]:
df_unlabeled = pd.DataFrame({'review':train_text[train_unlabeled_idxs], 'sentiment':train_labels[train_unlabeled_idxs]})
print(df_unlabeled.shape)
df_unlabeled.head()

(2000, 2)


,review,sentiment
0,"This is the epitome of bad 80's film-making, u...",0
1,(Only minor spoilers except as noted).<br /><b...,0
2,'Dead Letter Office' is a low-budget film abou...,0
3,"Anyone who visited drive-ins in the 1950s, 60s...",0
4,Feature of early 21 century cinema of lets pit...,0


In [ ]:
#Lemmatize the dataset
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
%%time
clean_train_df = clean(df_train)
clean_test_df = clean(df_test)
clean_unlabeled_df = clean(df_unlabeled)

CPU times: user 13.2 s, sys: 221 ms, total: 13.4 s
Wall time: 13.3 s


In [ ]:
# texts = (df_train['review'].append(df_test['review'], ignore_index=True)).apply(lambda x:x.split())
texts = (clean_train_df['review'].append(clean_unlabeled_df['review'], ignore_index=True)).apply(lambda x:x.split())

In [ ]:
%%time
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import word2vec
# Create Corpus
model = word2vec.Word2Vec(texts, min_count=1)

CPU times: user 15.6 s, sys: 199 ms, total: 15.8 s
Wall time: 10.8 s


In [ ]:
word_li=list(model.wv.vocab)
print(word_li[:5])

['okay', 'lets', 'cut', 'to', 'the']


In [ ]:
##save the modeled words produced from Word2Vec
model.save('word2vec_model.bin')

In [ ]:
from gensim.models import KeyedVectors
loaded_model=KeyedVectors.load('word2vec_model.bin')
print(loaded_model)

Word2Vec(vocab=39344, size=100, alpha=0.025)


In [ ]:
%%time
num_features = 100 # See Word2Vec(vocab=213802, size=100, alpha=0.025) in above cell.
feature_vec_train = get_avg_feature_vecs(clean_train_df['review'], loaded_model, num_features)
feature_vec_test = get_avg_feature_vecs(clean_test_df['review'], loaded_model, num_features)
feature_vec_unlabeled = get_avg_feature_vecs(clean_unlabeled_df['review'], loaded_model, num_features)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


CPU times: user 2min 7s, sys: 1.51 s, total: 2min 9s
Wall time: 2min 8s


In [ ]:
print(feature_vec_train.shape, feature_vec_test.shape, feature_vec_unlabeled.shape)

(400, 100) (12500, 100) (2000, 100)


In [ ]:
print(type(feature_vec_train), type(feature_vec_test), type(feature_vec_unlabeled))

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [ ]:
feature_vec_full_train = np.concatenate((feature_vec_train, feature_vec_unlabeled))
feature_vec_full_train.shape

(2400, 100)

In [ ]:
labels = np.array([i for i in list(df_train.sentiment)]+[-1 for i in list(df_unlabeled.sentiment)])
print(labels)
len(labels)

[ 0  0  0 ... -1 -1 -1]


2400

In [ ]:
#n_labeled=20
%%time
from sklearn.semi_supervised import LabelSpreading
label_prop_model = LabelSpreading()
label_prop_model.fit(feature_vec_full_train, labels)

from sklearn.metrics import accuracy_score
test_labels = np.array([i for i in list(df_test.sentiment)])
y_hat = label_prop_model.predict(feature_vec_test)
score = accuracy_score(test_labels, y_hat)
print('Accuracy: %.3f' % (score*100))

Accuracy: 49.592


In [ ]:
#n_labeled=400
%%time
from sklearn.semi_supervised import LabelSpreading
label_prop_model = LabelSpreading()
label_prop_model.fit(feature_vec_full_train, labels)

from sklearn.metrics import accuracy_score
test_labels = np.array([i for i in list(df_test.sentiment)])
y_hat = label_prop_model.predict(feature_vec_test)
score = accuracy_score(test_labels, y_hat)
print('Accuracy: %.3f' % (score*100))

Accuracy: 53.744
CPU times: user 3.31 s, sys: 1.82 s, total: 5.14 s
Wall time: 2.74 s


In [ ]:
#n_labeled=1000
%%time
from sklearn.semi_supervised import LabelSpreading
label_prop_model = LabelSpreading()
label_prop_model.fit(feature_vec_full_train, labels)

from sklearn.metrics import accuracy_score
test_labels = np.array([i for i in list(df_test.sentiment)])
y_hat = label_prop_model.predict(feature_vec_test)
score = accuracy_score(test_labels, y_hat)
print('Accuracy: %.3f' % (score*100))

In [ ]:
#n_labeled=2000
%%time
from sklearn.semi_supervised import LabelSpreading
label_prop_model = LabelSpreading()
label_prop_model.fit(feature_vec_full_train, labels)

from sklearn.metrics import accuracy_score
test_labels = np.array([i for i in list(df_test.sentiment)])
y_hat = label_prop_model.predict(feature_vec_test)
score = accuracy_score(test_labels, y_hat)
print('Accuracy: %.3f' % (score*100))

In [ ]:
#n_labeled=4800
%%time
from sklearn.semi_supervised import LabelSpreading
label_prop_model = LabelSpreading()
label_prop_model.fit(feature_vec_full_train, labels)

from sklearn.metrics import accuracy_score
test_labels = np.array([i for i in list(df_test.sentiment)])
y_hat = label_prop_model.predict(feature_vec_test)
score = accuracy_score(test_labels, y_hat)
print('Accuracy: %.3f' % (score*100))